## 2015 play by play data

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.cm as cm
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy
import zipfile
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import seaborn as sn
from sklearn.metrics import silhouette_samples, silhouette_score


sys.setrecursionlimit(100000)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

### import 2015 nhl data set

In [2]:
da = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_pbp20152016.csv')
#da = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_pbp20152016.csv')
da = da.drop('Unnamed: 0', axis=1)

- rename columns and create a season variable

In [4]:
da['Season'] = 2015
da = da.rename(columns={ 'Game_Id' : 'GameNumber', 'Ev_Zone': 'EventZone', 'Time_Elapsed': 'Time', 'Ev_Team': 'EventTeamCode', 'Description': 'EventDetail', 'Away_Team': 'ATeamCode', 'Home_Team':'HTeamCode', 'p1_name': 'EventPlayer1','p2_name': 'EventPlayer2', 'p3_name': 'EventPlayer3', 'awayPlayer1': 'APlayer1', 'awayPlayer2': 'APlayer2', 'awayPlayer3': 'APlayer3', 'awayPlayer4': 'APlayer4', 'awayPlayer5': 'APlayer5', 'awayPlayer6': 'APlayer6', 'homePlayer1': 'HPlayer1', 'homePlayer2': 'HPlayer2', 'homePlayer3': 'HPlayer3','homePlayer4': 'HPlayer4', 'homePlayer5': 'HPlayer5','homePlayer6': 'HPlayer6', 'Away_Score': 'AGoals', 'Home_Score': 'HGoals', 'Away_Goalie': 'AGoalie', 'Home_Goalie': 'HGoalie', 'Away_Players': 'PlayersA', 'Home_Players': 'PlayersH' })
da = da[['Season', 'GameNumber', 'Date', 'Period', 'Time', 'Strength', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'ATeamCode', 'APlayer1', 'APlayer2', 'APlayer3', 'APlayer4', 'APlayer5', 'APlayer6', 'AGoalie', 'PlayersA', 'HTeamCode', 'HPlayer1', 'HPlayer2', 'HPlayer3', 'HPlayer4', 'HPlayer5', 'HPlayer6', 'HGoalie', 'PlayersH']]
da = da.sort_values(['Season', 'GameNumber', 'Period', 'Time'], ascending=[True, True, True, True])

- keep only regular season games and exclude irrelevant on-ice events

In [5]:
da = da[da['GameNumber'] <= 21230]
da = da[da['Period'] <= 4]
da = da[da['Period'] >= 1]
da = da[da['Event']!='STOP']
da = da[da['Event']!='EISTR']
da = da[da['Event']!='EIEND']
da = da[da['Event']!='PSTR']
da = da[da['Event']!='PEND']
da = da[da['Event']!='SOC']
da = da[da['Event']!='GEND']

- create an event number variable that will count the number of events per game. 
- create an advantage type variable for even strength, power play and short handed situations.

In [6]:
da['EventNumber'] = da.groupby(['Season', 'GameNumber']).cumcount()+1

In [7]:
da['AdvantageType'] = da.apply(lambda x: 'EV' if ((x['PlayersA'] == x['PlayersH']) & (x['PlayersA'] != 1) & (x['PlayersH'] != 1)) else 'PP' if ((x['PlayersA'] > x['PlayersH']) & (x['EventTeamCode'] == x['ATeamCode'])) else 'PP' if ((x['PlayersA'] < x['PlayersH']) & (x['EventTeamCode'] == x['HTeamCode'])) else 'SH' if ((x['PlayersA'] < x['PlayersH']) & (x['EventTeamCode'] == x['ATeamCode'])) else 'SH' if ((x['PlayersA'] > x['PlayersH']) & (x['EventTeamCode'] == x['HTeamCode'])) else 'PP' if ((x['PlayersA'] == 1 ) & (x['PlayersH'] == 1)) else np.nan, axis=1) 

In [8]:
da['AdvantageType'].value_counts()

EV    264962
PP     33578
SH     14890
Name: AdvantageType, dtype: int64

- reshape data from wide to long for away team players and home team players

In [9]:
db = da.copy()
a = [col for col in db.columns if 'APlayer' in col]
b = [col for col in db.columns if 'HPlayer' in col]
db = pd.lreshape(db, {'APlayer' : a, 'HPlayer' : b })
db.head()

,AGoalie,ATeamCode,AdvantageType,Date,Event,EventDetail,EventNumber,EventPlayer1,EventPlayer2,EventPlayer3,EventTeamCode,GameNumber,HGoalie,HTeamCode,Period,PlayersA,PlayersH,Season,Strength,Time,APlayer,HPlayer
0,CAREY PRICE,MTL,EV,2015-10-07,FAC,MTL won Neu. Zone - MTL #14 PLEKANEC vs TOR #4...,1,TOMAS PLEKANEC,TYLER BOZAK,NaN,MTL,20001,JONATHAN BERNIER,TOR,1,6,6,2015,5x5,0:00,TOMAS PLEKANEC,SHAWN MATTHIAS
1,CAREY PRICE,MTL,EV,2015-10-07,FAC,MTL won Neu. Zone - MTL #14 PLEKANEC vs TOR #4...,2,TOMAS PLEKANEC,TYLER BOZAK,NaN,MTL,20001,JONATHAN BERNIER,TOR,1,6,6,2015,5x5,0:14,TOMAS PLEKANEC,SHAWN MATTHIAS
2,CAREY PRICE,MTL,EV,2015-10-07,HIT,"MTL #11 GALLAGHER HIT TOR #2 HUNWICK, Off. Zone",3,BRENDAN GALLAGHER,MATT HUNWICK,NaN,MTL,20001,JONATHAN BERNIER,TOR,1,6,6,2015,5x5,0:40,DAVID DESHARNAIS,SHAWN MATTHIAS
3,CAREY PRICE,MTL,EV,2015-10-07,SHOT,"TOR ONGOAL - #28 BOYES, Wrist, Off. Zone, 35 ft.",4,BRAD BOYES,NaN,NaN,TOR,20001,JONATHAN BERNIER,TOR,1,6,6,2015,5x5,0:51,DAVID DESHARNAIS,NAZEM KADRI
4,CAREY PRICE,MTL,EV,2015-10-07,GIVE,"MTL GIVEAWAY - #74 EMELIN, Def. Zone",5,ALEXEI EMELIN,NaN,NaN,MTL,20001,JONATHAN BERNIER,TOR,1,6,6,2015,5x5,0:58,DAVID DESHARNAIS,NAZEM KADRI


In [10]:
db = db[['Season', 'GameNumber', 'Date', 'Period', 'Time', 'AdvantageType', 'Strength', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'ATeamCode', 'APlayer', 'AGoalie', 'PlayersA', 'HTeamCode', 'HPlayer', 'HGoalie', 'PlayersH']]
db = db.sort_values(['Season', 'GameNumber', 'Period', 'Time'], ascending=[True, True, True, True])

- reshape data from wide to long for away and home team code and player respectfully.

In [11]:
dc = db.copy()
dc = dc.rename(columns={'EventTeamCode': 'EventTeam', 'EventPlayer1': 'EventP1', 'EventPlayer2': 'EventP2', 'EventPlayer3': 'EventP3', 'PlayersA':'PA', 'PlayersH': 'PH'})

In [12]:
a = [col for col in dc.columns if 'Code' in col]
b = [col for col in dc.columns if 'Player' in col]
c = [col for col in dc.columns if 'Goalie' in col]
d = [col for col in dc.columns if 'Position' in col]
dc = pd.lreshape(dc, {'TeamCode' : a, 'PlayerName' : b, 'Goalie': c})

In [13]:
dc = dc.rename(columns={'EventTeam': 'EventTeamCode', 'EventP1': 'EventPlayer1', 'EventP2': 'EventPlayer2', 'EventP3': 'EventPlayer3', 'PA':'PlayersA', 'PH': 'PlayersH'})

In [14]:
dc.shape

(3594286, 19)

### games played

- calculate the games each player participated in for the duration of the 2011 regular season.

In [15]:
s = dc.copy()
s = s[['Season', 'GameNumber', 'TeamCode', 'PlayerName']]
s = s.sort_values(['Season', 'GameNumber', 'TeamCode'], ascending=[True, True, True])
s = s.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])

In [16]:
s['GP'] = s.groupby(['Season', 'TeamCode','PlayerName'])['GameNumber'].transform('count')
s = s.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
s = s[['Season', 'TeamCode', 'PlayerName', 'GP']]
s.head()

,Season,TeamCode,PlayerName,GP
0,2015,MTL,TOMAS PLEKANEC,82
1,2015,MTL,BRENDAN GALLAGHER,53
2,2015,MTL,MAX PACIORETTY,82
3,2015,MTL,PK SUBBAN,68
4,2015,MTL,ANDREI MARKOV,82


In [17]:
s.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
dtype: int64

In [18]:
s.shape

(1065, 4)

### shots for and against

- calculate the shots against each player faced througout the duration of a season. 
- create shot for and shot against variables. If event team code is indiferrent to team code assign a value of 1. If not, assign a value of 0. 
- **goals are a result of a shot and therefore need to be added to a player's shot total**. Display total shots per player for a season.

In [19]:
sa = dc.copy()
sa = sa[sa['Event'].isin(['SHOT','GOAL'])]

In [20]:
sa['ShotF'] = sa.apply(lambda x: 1 if x['EventTeamCode'] == x['TeamCode'] else 0, axis=1)
sa['ShotsF'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['ShotF'].transform('sum')

sa['ShotA'] = sa.apply(lambda x: 1 if x['EventTeamCode'] != x['TeamCode'] else 0, axis=1)
sa['ShotsA'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['ShotA'].transform('sum')

- create shot for and shot against variables for even strength situations only. If event team code is indiferrent to team code assign a value of 1. If not, assign a value of 0. 
- **goals are a result of a shot and therefore need to be added to a player's shot total**. Display total shots per player for a season.

In [21]:
sa['EVShotF'] = sa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] == x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] == x['TeamCode'])) else 0, axis=1)
sa['EVShotsF'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVShotF'].transform('sum')

sa['EVShotA'] = sa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] != x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] != x['TeamCode'])) else 0, axis=1)
sa['EVShotsA'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVShotA'].transform('sum')

- calculate the shot differential for all shots and even strength shots.

In [22]:
sa['DShots'] = sa['ShotsF'] - sa['ShotsA']
sa['EVDShots'] = sa['EVShotsF'] - sa['EVShotsA']

In [23]:
sa = sa [['Season', 'TeamCode', 'PlayerName', 'ShotsF', 'ShotsA', 'DShots', 'EVShotsF', 'EVShotsA', 'EVDShots' ]]
sa = sa.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
sa.head()

,Season,TeamCode,PlayerName,ShotsF,ShotsA,DShots,EVShotsF,EVShotsA,EVDShots
18,2015,MTL,DAVID DESHARNAIS,555,445,110,435,444,-9
19,2015,MTL,DALE WEISE,448,344,104,364,341,23
20,2015,MTL,TOMAS FLEISCHMANN,390,444,-54,362,383,-21
21,2015,MTL,JEFF PETRY,523,492,31,460,397,63
22,2015,MTL,ALEXEI EMELIN,667,750,-83,661,612,49


In [24]:
sa.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
ShotsF        0
ShotsA        0
DShots        0
EVShotsF      0
EVShotsA      0
EVDShots      0
dtype: int64

In [25]:
sa.shape

(1064, 9)

### goals for and against

- create goals for and goals against variable. If event team code is the same as team code assign a value of 1 for goals for and a value of 0 for goals against. If event team code is different to team code, assign a value of 0 for golas for and a value of 1 for goals against. Display total goals for and against per player for a season.

In [26]:
gfa = dc.copy()
gfa = gfa[gfa['Event'] == 'GOAL']

In [27]:
gfa['EVGoalF'] = gfa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] == x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] == x['TeamCode'])) else 0, axis=1)
gfa['EVGoalA'] = gfa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] != x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] != x['TeamCode'])) else 0, axis=1)

In [28]:
gfa['GoalF'] = gfa.apply(lambda x: 1 if (x['EventTeamCode'] == x['TeamCode']) else 0, axis=1)
gfa['GoalA'] = gfa.apply(lambda x: 1 if (x['EventTeamCode'] != x['TeamCode']) else 0, axis=1)

In [29]:
gfa['EVGoalsF'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVGoalF'].transform('sum')
gfa['EVGoalsA'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVGoalA'].transform('sum')
gfa['GoalsF'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['GoalF'].transform('sum')
gfa['GoalsA'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['GoalA'].transform('sum')

In [30]:
gfa = gfa[['Season', 'GameNumber', 'TeamCode', 'PlayerName' , 'GoalsF', 'GoalsA', 'EVGoalsF', 'EVGoalsA']]
gfa = gfa.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
gfa['Plus/Minus'] = gfa['EVGoalsF'] - gfa['EVGoalsA']
gfa = gfa.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
gfa = gfa[['Season', 'TeamCode', 'PlayerName' , 'GoalsF', 'GoalsA', 'EVGoalsF', 'EVGoalsA', 'Plus/Minus']]
gfa.head()

,Season,TeamCode,PlayerName,GoalsF,GoalsA,EVGoalsF,EVGoalsA,Plus/Minus
341,2015,MTL,TOMAS PLEKANEC,83,61,58,45,13
343,2015,MTL,BRENDAN GALLAGHER,65,25,44,25,19
345,2015,MTL,MAX PACIORETTY,88,73,61,62,-1
347,2015,MTL,PK SUBBAN,107,75,72,60,12
349,2015,MTL,ANDREI MARKOV,98,85,67,64,3


In [31]:
gfa.shape

(1033, 8)

### create dataframes for all on-ice event types

#### goals dataframe (dg)

In [32]:
dg = dc.copy()
dg = dg[dg['Event'] == 'GOAL']

In [33]:
dg = dg[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dg = dg.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a goal variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total goals per player for a season.

In [34]:
dg['Goal'] = dg.apply(lambda x: 1 if (x['Event'] == 'GOAL') else 0, axis=1)
dg = dg.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dg['Goals'] = dg.groupby(['Season','TeamCode', 'PlayerName'])['Goal'].transform('sum')

- keep one observation per player by game and drop duplicates. This purpose of this step is to calculate the quantity of games per player for a season. 

In [35]:
dg = dg.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dg = dg.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dg = dg[['Season', 'TeamCode', 'PlayerName', 'Goals']]
dg.head()

,Season,TeamCode,PlayerName,Goals
341,2015,MTL,MAX PACIORETTY,30
579,2015,TOR,JAMES VAN RIEMSDYK,14
1177,2015,MTL,ALEX GALCHENYUK,30
1728,2015,CHI,ARTEMI PANARIN,30
1842,2015,NYR,DEREK STEPAN,22


In [36]:
dg.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Goals         0
dtype: int64

In [37]:
dg.shape

(728, 4)

#### assists dataframe (dast)

In [38]:
dast = dc.copy()
dast = dast[dast['Event'] == 'GOAL']

In [39]:
dast = dast[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer2', 'EventPlayer3']]
dast = dast.rename(columns={'EventPlayer2': 'Assist1Name', 'EventPlayer3': 'Assist2Name', 'EventTeamCode':'TeamCode'})

- reshape assist players 1 and 2 to create one column of assist players

In [40]:
a = [col for col in dast.columns if 'Name' in col]
dast = pd.lreshape(dast, {'PlayerName' : a})

- create an assist variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total goals per player for a season.

In [41]:
dast['Assist'] = dast.apply(lambda x: 1 if (x['Event'] == 'GOAL') else 0, axis=1)
dast = dast.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dast['Assists'] = dast.groupby(['Season','TeamCode', 'PlayerName'])['Assist'].transform('sum')

- keep one observation per player by game and drop duplicates. This purpose of this step is to calculate the quantity of games per player for a season. 

In [42]:
dast = dast.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dast = dast.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dast = dast[['Season', 'TeamCode', 'PlayerName', 'Assists']]
dast.head()

,Season,TeamCode,PlayerName,Assists
0,2015,MTL,PK SUBBAN,45
6,2015,TOR,NAZEM KADRI,28
11,2015,MTL,ANDREI MARKOV,39
23,2015,CHI,PATRICK KANE,60
29,2015,NYR,KEVIN HAYES,22


In [43]:
dast.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Assists       0
dtype: int64

In [44]:
dast.shape

(846, 4)

#### shot dataframe (ds)

In [45]:
ds = dc.copy()
ds = ds[ds['Event'] == 'SHOT']

In [46]:
ds = ds[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
ds = ds.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})
ds.head()

,Season,GameNumber,EventNumber,Event,TeamCode,PlayerName
18,2015,20001,4,SHOT,TOR,BRAD BOYES
19,2015,20001,4,SHOT,TOR,BRAD BOYES
20,2015,20001,4,SHOT,TOR,BRAD BOYES
21,2015,20001,4,SHOT,TOR,BRAD BOYES
22,2015,20001,4,SHOT,TOR,BRAD BOYES


- create a shot variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [47]:
ds['Shot'] = ds.apply(lambda x: 1 if (x['Event'] == 'SHOT') else 0, axis=1)
ds = ds.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
ds['Shots'] = ds.groupby(['Season','TeamCode', 'PlayerName'])['Shot'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [48]:
ds = ds.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
ds = ds.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
ds = ds[['Season', 'TeamCode', 'PlayerName', 'Shots']]
ds.head()

,Season,TeamCode,PlayerName,Shots
18,2015,TOR,BRAD BOYES,78
54,2015,MTL,TOMAS PLEKANEC,175
191,2015,TOR,MATT HUNWICK,72
209,2015,TOR,DION PHANEUF,113
227,2015,MTL,MAX PACIORETTY,273


In [49]:
ds.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Shots         0
dtype: int64

In [50]:
ds.shape

(942, 4)

#### block dataframe (db)

In [51]:
dbl = dc.copy()
dbl = dbl[dbl['Event'] == 'BLOCK']

In [52]:
dbl.shape

(397434, 19)

In [53]:
dbl['EventPlayer2'].isnull().sum()

0

In [54]:
dbl['EventPlayer3'].isnull().sum()

397434

In [55]:
dbl['BlockTeam'] = dbl.EventDetail.str.split(' ', expand = True)[6]
dbl['BlockTeamCode'] = dbl.BlockTeam.str.split(' ', expand = True)[0]

In [56]:
dbl = dbl[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventPlayer1', 'BlockTeamCode']]
dbl = dbl.rename(columns={'EventPlayer1': 'PlayerName', 'BlockTeamCode':'TeamCode'})

- create a block variable. If event type is a block assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [57]:
dbl['Block'] = dbl.apply(lambda x: 1 if (x['Event'] == 'BLOCK') else 0, axis=1)
dbl = dbl.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dbl['Blocks'] = dbl.groupby(['Season','TeamCode', 'PlayerName'])['Block'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [58]:
dbl = dbl.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dbl = dbl.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dbl = dbl[['Season', 'TeamCode', 'PlayerName', 'Blocks']]
dbl.head()

,Season,TeamCode,PlayerName,Blocks
72,2015,MTL,BRENDAN GALLAGHER,24
126,2015,MTL,NATHAN BEAULIEU,89
161,2015,TOR,PETER HOLLAND,19
167,2015,TOR,LEO KOMAROV,27
179,2015,TOR,JAKE GARDINER,66


In [59]:
dbl.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Blocks        0
dtype: int64

In [60]:
dbl.shape

(1139, 4)

#### hit dataframe (dh)

In [61]:
dhit = dc.copy()
dhit = dhit[dhit['Event'] == 'HIT']

In [62]:
dhit.shape

(674608, 19)

In [63]:
dhit['EventPlayer2'].isnull().sum()

0

In [64]:
dhit['EventPlayer3'].isnull().sum()

674608

In [65]:
dhit = dhit[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dhit = dhit.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a hit variable. If event type is a hit assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [66]:
dhit['Hit'] = dhit.apply(lambda x: 1 if (x['Event'] == 'HIT') else 0, axis=1)
dhit = dhit.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dhit['Hits'] = dhit.groupby(['Season','TeamCode', 'PlayerName'])['Hit'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [67]:
dhit = dhit.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dhit = dhit.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dhit = dhit[['Season', 'TeamCode', 'PlayerName', 'Hits']]
dhit.head()

,Season,TeamCode,PlayerName,Hits
12,2015,MTL,BRENDAN GALLAGHER,43
30,2015,TOR,PETER HOLLAND,78
42,2015,TOR,DANIEL WINNIK,41
84,2015,MTL,JEFF PETRY,86
121,2015,TOR,MATT HUNWICK,127


In [68]:
dhit.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Hits          0
dtype: int64

In [69]:
dhit.shape

(992, 4)

#### penalty dataframe (dpen)

In [70]:
dp = dc.copy()
dp = dp[dp['Event'] == 'PENL']

- use event detail to find the duration of a given penalty and to assign it to the proper player. Major penalty is 5 minutes, so a value of 5 is assigned for every event that had a major penalty.

In [72]:
dp['Pen'] = dp.EventDetail.str.split(')', expand = True)[0]
dp['Penal'] = dp.Pen.str.split('(', expand = True)[1]
dp['Penalt'] = dp.Penal.str.split(' ', expand = True)[0]
dp['Penalty'] = dp.apply(lambda x: 5 if x['Penalt'] == 'maj' else x['Penalt'], axis=1)
dp['Penalty'] = dp['Penalty'].convert_objects(convert_numeric=True)
dp = dp[['Season', 'GameNumber', 'Period', 'Time', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'TeamCode', 'PlayerName', 'Penalty']]

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


- use event detail to find penalty team code and penalty player name.

In [73]:
dp['PenaltyTeamCode'] = dp.EventDetail.str.split('\\s', expand = True)[0]
dp['PN'] = dp.EventDetail.str.split('\\s', expand = True)[1]
dp['PNumber'] = dp.PN.str.split('#', expand = True)[1]
dp['PenaltyPlayerNumber'] = dp.PNumber.str.split(' ', expand = True)[0]
dp['PenaltyPlayerLName'] = dp.EventDetail.str.split('\\s', expand = True)[2]
#dp['PenaltyPlayerLName'] = dp.PPlLN.str.split(' ', expand = True)[0]
dp = dp[['Season', 'GameNumber', 'Period', 'Time', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'TeamCode', 'PlayerName', 'Penalty', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyPlayerLName']]

- seperate player name into first and last. The purpose is to connect the correct player name to penalty player name.

In [74]:
dp['PlayerFName'] = dp.PlayerName.str.split('\\s', expand = True)[0]
dp['PlayerLName'] = dp.PlayerName.str.split('\\s', expand = True)[1]
dp['PenaltyPlayerFName'] = dp.apply(lambda x: x['PlayerFName'] if ((x['PenaltyTeamCode'] == x['TeamCode']) & (x['PenaltyPlayerNumber'] == x['PenaltyPlayerNumber']) & (x['PlayerLName'] == x['PenaltyPlayerLName'])) else np.nan, axis=1)
dp['PenaltyPlayerFName'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyPlayerLName'])['PenaltyPlayerFName'].apply(lambda x: x.ffill().bfill())

In [75]:
dp['PenaltyName'] = dp.apply(lambda x: x['PlayerName'] if ((x['PenaltyTeamCode'] == x['TeamCode']) & (x['PenaltyPlayerNumber'] == x['PenaltyPlayerNumber']) & (x['PlayerLName'] == x['PenaltyPlayerLName']) & (x['PlayerFName'] == x['PenaltyPlayerFName'])) else np.nan, axis=1)
dp['PenaltyName'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerLName', 'PenaltyPlayerNumber', 'PenaltyPlayerLName'])['PenaltyName'].apply(lambda x: x.ffill().bfill())


- drop duplicates per game and event number. Calculate the total number of penalties for each player.

In [76]:
dp = dp.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyName'])
dp['Penalties'] = dp.groupby(['Season', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyName'])['Penalty'].transform('sum')
dp['Penalties'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerNumber', 'PlayerName'])['Penalties'].apply(lambda x: x.ffill().bfill())
#dp.head()

- drop duplicates to keep one observation per player for the season.

In [77]:
dp = dp[['Season', 'PenaltyTeamCode', 'PenaltyName', 'Penalties']]
dp = dp.drop_duplicates(['Season', 'PenaltyTeamCode', 'PenaltyName'])
dp = dp.rename(columns={'PenaltyTeamCode':'TeamCode', 'PenaltyName':'PlayerName'})
dp = dp[['Season', 'TeamCode', 'PlayerName', 'Penalties']]
dp.head()

,Season,TeamCode,PlayerName,Penalties
90,2015,TOR,NAZEM KADRI,73.0
408,2015,TOR,LEO KOMAROV,40.0
409,2015,MTL,DEVANTE SMITH-PELLY,22.0
568,2015,MTL,BRENDAN GALLAGHER,24.0
1088,2015,MTL,LARS ELLER,26.0


- keep observations that are not null. Since a team can be allocated a penalty, those observations are excluded from the data.

In [78]:
dp['PlayerName'].isnull().sum()

30

In [79]:
dp = dp[dp['PlayerName'].notnull()]

In [80]:
dp.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Penalties     0
dtype: int64

In [81]:
dp.shape

(889, 4)

####  miss dataframe (dmiss)

In [82]:
dmiss = dc.copy()
dmiss = dmiss[dmiss['Event'] == 'MISS']

In [83]:
dmiss.shape

(324756, 19)

In [84]:
dmiss['EventPlayer2'].isnull().sum()

324756

In [85]:
dmiss['EventPlayer3'].isnull().sum()

324756

In [86]:
dmiss = dmiss[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dmiss = dmiss.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a miss variable. If event type is a miss assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [87]:
dmiss['Miss'] = dmiss.apply(lambda x: 1 if (x['Event'] == 'MISS') else 0, axis=1)
dmiss = dmiss.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dmiss['Misses'] = dmiss.groupby(['Season','TeamCode', 'PlayerName'])['Miss'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [88]:
dmiss = dmiss.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dmiss = dmiss.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dmiss = dmiss[['Season', 'TeamCode', 'PlayerName', 'Misses']]
dmiss.head()

,Season,TeamCode,PlayerName,Misses
66,2015,MTL,PK SUBBAN,72
101,2015,MTL,ANDREI MARKOV,58
111,2015,MTL,ALEX GALCHENYUK,80
239,2015,MTL,LARS ELLER,60
299,2015,TOR,SCOTT HARRINGTON,8


In [89]:
dmiss.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Misses        0
dtype: int64

In [90]:
dmiss.shape

(899, 4)

####  takeaway dataframe (dtake)

In [91]:
dtake = dc.copy()
dtake = dtake[dtake['Event'] == 'TAKE']

In [92]:
dtake.shape

(178901, 19)

In [93]:
dtake['EventPlayer2'].isnull().sum()

178901

In [94]:
dtake['EventPlayer3'].isnull().sum()

178901

In [95]:
dtake = dtake[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dtake = dtake.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a takeaway variable. If event type is a takeaway assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [96]:
dtake['Takeaway'] = dtake.apply(lambda x: 1 if (x['Event'] == 'TAKE') else 0, axis=1)
dtake = dtake.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dtake['Takeaways'] = dtake.groupby(['Season','TeamCode', 'PlayerName'])['Takeaway'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [97]:
dtake = dtake.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dtake = dtake.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dtake = dtake[['Season', 'TeamCode', 'PlayerName', 'Takeaways']]
dtake.head()

,Season,TeamCode,PlayerName,Takeaways
143,2015,TOR,BRAD BOYES,25
185,2015,TOR,SCOTT HARRINGTON,1
221,2015,MTL,BRENDAN GALLAGHER,30
245,2015,MTL,ALEXANDER SEMIN,6
465,2015,TOR,MATT HUNWICK,24


In [98]:
dtake.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Takeaways     0
dtype: int64

In [99]:
dtake.shape

(868, 4)

####  giveaway dataframe (dgive)

In [100]:
dgive = dc.copy()
dgive = dgive[dgive['Event'] == 'GIVE']

In [101]:
dgive.shape

(235069, 19)

In [102]:
dgive['EventPlayer2'].isnull().sum()

235069

In [103]:
dgive['EventPlayer3'].isnull().sum()

235069

In [104]:
dgive = dgive[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dgive = dgive.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a giveaway variable. If event type is a giveaway assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [105]:
dgive['Giveaway'] = dgive.apply(lambda x: 1 if (x['Event'] == 'GIVE') else 0, axis=1)
dgive = dgive.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dgive['Giveaways'] = dgive.groupby(['Season','TeamCode', 'PlayerName'])['Giveaway'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [106]:
dgive = dgive.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dgive = dgive.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dgive = dgive[['Season', 'TeamCode', 'PlayerName', 'Giveaways']]
dgive.head()

,Season,TeamCode,PlayerName,Giveaways
24,2015,MTL,ALEXEI EMELIN,75
48,2015,TOR,MORGAN RIELLY,72
131,2015,MTL,NATHAN BEAULIEU,48
311,2015,MTL,TORREY MITCHELL,27
330,2015,MTL,DEVANTE SMITH-PELLY,15


In [107]:
dgive.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Giveaways     0
dtype: int64

In [108]:
dgive.shape

(965, 4)

### import 2015 player shifts

In [109]:
dshift = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_shifts20152016.csv')
#dshift = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_shifts20152016.csv')
dshift = dshift.drop('Unnamed: 0', axis=1)

- rename variables, keep only regular season games and create a season variable.

In [110]:
dshift['Season'] = 2015
dshift= dshift.rename(columns={ 'Game_Id' : 'GameNumber', 'Team': 'TeamCode', 'Player': 'PlayerName' })
dshift = dshift[dshift['GameNumber'] <= 21230]
dshift = dshift[['Season', 'Date', 'GameNumber', 'Period', 'TeamCode', 'PlayerName', 'Player_Id', 'Start', 'End', 'Duration']]
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration
0,2015,2015-10-07,20001,1,MTL,ANDREI MARKOV,8467496.0,0.0,36.0,36.0
1,2015,2015-10-07,20001,1,MTL,MAX PACIORETTY,8474157.0,0.0,36.0,36.0
2,2015,2015-10-07,20001,1,MTL,TOMAS PLEKANEC,8469521.0,0.0,36.0,36.0
3,2015,2015-10-07,20001,1,TOR,JOFFREY LUPUL,8470207.0,0.0,37.0,37.0
4,2015,2015-10-07,20001,1,TOR,TYLER BOZAK,8475098.0,0.0,37.0,37.0


- calcuate the seconds an individual played per game. Convert seconds to minutes and drop duplicates per season, game number, team and player.

In [111]:
dshift['Seconds'] = dshift.groupby(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])['Duration'].transform('sum')
dshift['Minutes'] = dshift['Seconds']/60
dshift['Minutes'] = dshift['Minutes'].round(2)

In [112]:
dshift = dshift.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])

In [113]:
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration,Seconds,Minutes
0,2015,2015-10-07,20001,1,MTL,ANDREI MARKOV,8467496.0,0.0,36.0,36.0,1304.0,21.73
1,2015,2015-10-07,20001,1,MTL,MAX PACIORETTY,8474157.0,0.0,36.0,36.0,1087.0,18.12
2,2015,2015-10-07,20001,1,MTL,TOMAS PLEKANEC,8469521.0,0.0,36.0,36.0,1109.0,18.48
3,2015,2015-10-07,20001,1,TOR,JOFFREY LUPUL,8470207.0,0.0,37.0,37.0,1064.0,17.73
4,2015,2015-10-07,20001,1,TOR,TYLER BOZAK,8475098.0,0.0,37.0,37.0,1143.0,19.05


- calcuate the seconds an individual played for the whole regular season. Convert seconds to minutes and drop duplicates per season, game number, team and player.

In [114]:
dshift['TSeconds'] = dshift.groupby(['Season', 'TeamCode', 'PlayerName'])['Seconds'].transform('sum')
dshift['TOI'] = dshift['TSeconds']/60
dshift['TOI'] = dshift['TOI'].round(0)
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration,Seconds,Minutes,TSeconds,TOI
0,2015,2015-10-07,20001,1,MTL,ANDREI MARKOV,8467496.0,0.0,36.0,36.0,1304.0,21.73,117272.0,1955.0
1,2015,2015-10-07,20001,1,MTL,MAX PACIORETTY,8474157.0,0.0,36.0,36.0,1087.0,18.12,91154.0,1519.0
2,2015,2015-10-07,20001,1,MTL,TOMAS PLEKANEC,8469521.0,0.0,36.0,36.0,1109.0,18.48,91178.0,1520.0
3,2015,2015-10-07,20001,1,TOR,JOFFREY LUPUL,8470207.0,0.0,37.0,37.0,1064.0,17.73,40354.0,673.0
4,2015,2015-10-07,20001,1,TOR,TYLER BOZAK,8475098.0,0.0,37.0,37.0,1143.0,19.05,59308.0,988.0


- keep only observation per player for the season

In [115]:
dshift = dshift.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dshift = dshift[['Season', 'TeamCode', 'PlayerName', 'TOI']]
dshift.head()

,Season,TeamCode,PlayerName,TOI
0,2015,MTL,ANDREI MARKOV,1955.0
1,2015,MTL,MAX PACIORETTY,1519.0
2,2015,MTL,TOMAS PLEKANEC,1520.0
3,2015,TOR,JOFFREY LUPUL,673.0
4,2015,TOR,TYLER BOZAK,988.0


In [116]:
dshift.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
TOI           0
dtype: int64

In [117]:
dshift.shape

(1065, 4)

### import 2015 player positions

In [131]:
dpos = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_positions20152016.csv')
#dpos = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_positions20152015.csv')

In [132]:
dpos.head()

,FirstName,LastName,TeamCode,Position,FName,LName,PlayerName
0,Justin,Abdelkader,DET,LW/RW/C,JUSTIN,ABDELKADER,JUSTIN ABDELKADER
1,Noel,Acciari,BOS,C,NOEL,ACCIARI,NOEL ACCIARI
2,Kenny,Agostino,CGY,LW,KENNY,AGOSTINO,KENNY AGOSTINO
3,Andrew,Agozzino,COL,LW/C,ANDREW,AGOZZINO,ANDREW AGOZZINO
4,Karl,Alzner,WSH,D,KARL,ALZNER,KARL ALZNER


In [133]:
dpos['Season'] = 2015
dpos = dpos[['Season', 'TeamCode', 'PlayerName', 'Position']]
dpos.head()

,Season,TeamCode,PlayerName,Position
0,2015,DET,JUSTIN ABDELKADER,LW/RW/C
1,2015,BOS,NOEL ACCIARI,C
2,2015,CGY,KENNY AGOSTINO,LW
3,2015,COL,ANDREW AGOZZINO,LW/C
4,2015,WSH,KARL ALZNER,D


In [134]:
dpos.dtypes

Season         int64
TeamCode      object
PlayerName    object
Position      object
dtype: object

In [135]:
dpos['Position'] = dpos.apply(lambda x: 'C' if x['Position'] == 'C/LW' else 'C' if x['Position'] == 'C/RW' else 'C' if x['Position'] == 'C/RW/LW' else 'C' if x['Position'] == 'C/LW/RW' else 'W' if x['Position'] == 'RW' else 'W' if x['Position'] == 'LW' else 'W' if x['Position'] == 'LW/C' else 'W' if x['Position'] == 'RW/C' else 'W' if x['Position'] == 'RW/LW' else 'W' if x['Position'] == 'LW/RW' else 'W' if x['Position'] == 'RW/LW/C' else 'W' if x['Position'] == 'RW/C/LW' else 'W' if x['Position'] == 'RW/C/LW' else 'W' if x['Position'] == 'LW/D' else 'W' if x['Position'] == 'LW/C/RW' else 'W' if x['Position'] == 'LW/RW/C' else 'D' if x['Position'] == 'D/LW' else 'D' if x['Position'] == 'D/RW' else 'C' if x['Position'] == 'C/D' else 'W' if x['Position'] == 'RW/D' else 'C' if x['Position'] == 'C/N' else x['Position'] , axis=1)
dpos['Position'].value_counts()

W    333
D    306
C    259
G     92
Name: Position, dtype: int64

In [136]:
dpos['TeamCode'] = dpos.apply(lambda x: 'L.A' if x['TeamCode'] == 'LAK' else 'N.J' if x['TeamCode'] == 'NJD' else 'S.J' if x['TeamCode'] == 'SJS' else 'T.B' if x['TeamCode'] == 'TBL' else x['TeamCode'] , axis=1)
dpos.head()

,Season,TeamCode,PlayerName,Position
0,2015,DET,JUSTIN ABDELKADER,W
1,2015,BOS,NOEL ACCIARI,C
2,2015,CGY,KENNY AGOSTINO,W
3,2015,COL,ANDREW AGOZZINO,W
4,2015,WSH,KARL ALZNER,D


In [137]:
dpos.shape

(990, 4)

### merge data frames

- merge games played, goals and assists.

In [138]:
dd = pd.merge(s, dpos, on=['Season', 'TeamCode', 'PlayerName'], how='left').merge(dg,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dast,on=['Season', 'TeamCode', 'PlayerName'], how = 'left')

- forward and backward fill players that have been traded during the season. Fill in positions for three players.

In [139]:
dd['Position'] = dd.groupby(['PlayerName'])['Position'].apply(lambda x: x.ffill().bfill())
dd.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists
0,2015,MTL,TOMAS PLEKANEC,82,C,14.0,40.0
1,2015,MTL,BRENDAN GALLAGHER,53,W,19.0,21.0
2,2015,MTL,MAX PACIORETTY,82,W,30.0,34.0
3,2015,MTL,PK SUBBAN,68,D,6.0,45.0
4,2015,MTL,ANDREI MARKOV,82,D,5.0,39.0


In [140]:
dd.isnull().sum()

Season          0
TeamCode        0
PlayerName      0
GP              0
Position        0
Goals         337
Assists       219
dtype: int64

In [141]:
fix = dd.copy()
fix = fix[fix['Position'].isnull()]
fix.head(20)

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists


- fill in goals and assists with 0 for players who had no points.

In [142]:
dd['Goals'] = dd['Goals'].fillna(0)
dd['Assists'] = dd['Assists'].fillna(0)
dd['Points'] = dd['Goals'] + dd['Assists']
dd.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists,Points
0,2015,MTL,TOMAS PLEKANEC,82,C,14.0,40.0,54.0
1,2015,MTL,BRENDAN GALLAGHER,53,W,19.0,21.0,40.0
2,2015,MTL,MAX PACIORETTY,82,W,30.0,34.0,64.0
3,2015,MTL,PK SUBBAN,68,D,6.0,45.0,51.0
4,2015,MTL,ANDREI MARKOV,82,D,5.0,39.0,44.0


In [143]:
dd.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
Position      0
Goals         0
Assists       0
Points        0
dtype: int64

In [144]:
de = pd.merge(dd, gfa,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(ds,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(sa,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dbl,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dhit,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dp,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dtake,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dgive,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dmiss,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dshift,on=['Season', 'TeamCode', 'PlayerName'], how = 'left')
de.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists,Points,GoalsF,GoalsA,EVGoalsF,EVGoalsA,Plus/Minus,Shots,ShotsF,ShotsA,DShots,EVShotsF,EVShotsA,EVDShots,Blocks,Hits,Penalties,Takeaways,Giveaways,Misses,TOI
0,2015,MTL,TOMAS PLEKANEC,82,C,14.0,40.0,54.0,83.0,61.0,58.0,45.0,13.0,175.0,800.0,711.0,89.0,618.0,602.0,16.0,46.0,60.0,36.0,15.0,45.0,61.0,1520.0
1,2015,MTL,BRENDAN GALLAGHER,53,W,19.0,21.0,40.0,65.0,25.0,44.0,25.0,19.0,154.0,555.0,379.0,176.0,420.0,378.0,42.0,24.0,43.0,24.0,30.0,14.0,42.0,879.0
2,2015,MTL,MAX PACIORETTY,82,W,30.0,34.0,64.0,88.0,73.0,61.0,62.0,-1.0,273.0,897.0,691.0,206.0,703.0,611.0,92.0,37.0,100.0,34.0,35.0,33.0,122.0,1519.0
3,2015,MTL,PK SUBBAN,68,D,6.0,45.0,51.0,107.0,75.0,72.0,60.0,12.0,170.0,1001.0,856.0,145.0,725.0,718.0,7.0,79.0,101.0,75.0,23.0,106.0,72.0,1793.0
4,2015,MTL,ANDREI MARKOV,82,D,5.0,39.0,44.0,98.0,85.0,67.0,64.0,3.0,112.0,992.0,992.0,0.0,754.0,824.0,-70.0,142.0,57.0,38.0,38.0,93.0,58.0,1955.0


- fill in with 0 for all nan values since not all players have been on the ice for a goal for or a goal against. 

In [145]:
de['GoalsF'] = de['GoalsF'].fillna(0)
de['GoalsA'] = de['GoalsA'].fillna(0)
de['EVGoalsF'] = de['EVGoalsF'].fillna(0)
de['EVGoalsA'] = de['EVGoalsA'].fillna(0)
de['Plus/Minus'] = de['Plus/Minus'].fillna(0)
de['Shots'] = de['Shots'].fillna(0)
de['Hits'] = de['Hits'].fillna(0)
de['Blocks'] = de['Blocks'].fillna(0)
de['Penalties'] = de['Penalties'].fillna(0)
de['Giveaways'] = de['Giveaways'].fillna(0)
de['Takeaways'] = de['Takeaways'].fillna(0)
de['Misses'] = de['Misses'].fillna(0)

In [146]:
de.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
Position      0
Goals         0
Assists       0
Points        0
GoalsF        0
GoalsA        0
EVGoalsF      0
EVGoalsA      0
Plus/Minus    0
Shots         0
ShotsF        1
ShotsA        1
DShots        1
EVShotsF      1
EVShotsA      1
EVDShots      1
Blocks        0
Hits          0
Penalties     0
Takeaways     0
Giveaways     0
Misses        0
TOI           0
dtype: int64

### total player stats for the season

- players that have been traded need to be evaluated for their season performance. Group by sesaon and player name to calcute the sum of each stat. 

In [147]:
de['TGP'] = de.groupby(['Season', 'PlayerName'])['GP'].transform('sum')
de['TGoals'] = de.groupby(['Season', 'PlayerName'])['Goals'].transform('sum')
de['TAssists'] = de.groupby(['Season', 'PlayerName'])['Assists'].transform('sum')
de['TPoints'] = de.groupby(['Season', 'PlayerName'])['Points'].transform('sum')
de['TGoalsF'] = de.groupby(['Season', 'PlayerName'])['GoalsF'].transform('sum')
de['TGoalsA'] = de.groupby(['Season', 'PlayerName'])['GoalsA'].transform('sum')
de['TEVGoalsF'] = de.groupby(['Season', 'PlayerName'])['EVGoalsF'].transform('sum')
de['TEVGoalsA'] = de.groupby(['Season', 'PlayerName'])['EVGoalsA'].transform('sum')
de['TPlus/Minus'] = de.groupby(['Season', 'PlayerName'])['Plus/Minus'].transform('sum')
de['TShots'] = de.groupby(['Season', 'PlayerName'])['Shots'].transform('sum')
de['TShotsF'] = de.groupby(['Season', 'PlayerName'])['ShotsF'].transform('sum')
de['TShotsA'] = de.groupby(['Season', 'PlayerName'])['ShotsA'].transform('sum')
de['TDShots'] = de.groupby(['Season', 'PlayerName'])['DShots'].transform('sum')
de['TEVShotsF'] = de.groupby(['Season', 'PlayerName'])['EVShotsF'].transform('sum')
de['TEVShotsA'] = de.groupby(['Season', 'PlayerName'])['EVShotsA'].transform('sum')
de['TEVDShots'] = de.groupby(['Season', 'PlayerName'])['EVDShots'].transform('sum')
de['TBlocks'] = de.groupby(['Season', 'PlayerName'])['Blocks'].transform('sum')
de['THits'] = de.groupby(['Season', 'PlayerName'])['Hits'].transform('sum')
de['TPenalties'] = de.groupby(['Season', 'PlayerName'])['Penalties'].transform('sum')
de['TTakeaways'] = de.groupby(['Season', 'PlayerName'])['Takeaways'].transform('sum')
de['TGiveaways'] = de.groupby(['Season', 'PlayerName'])['Giveaways'].transform('sum')
de['TMisses'] = de.groupby(['Season', 'PlayerName'])['Misses'].transform('sum')
de['TTOI'] = de.groupby(['Season', 'PlayerName'])['TOI'].transform('sum')

- divide all stats with total time on ice to scale the data for cluster analysis

In [148]:
de['TOIGoals'] = de['TGoals'] / de['TTOI']
de['TOIAssists'] = de['TAssists'] / de['TTOI']
de['TOIPoints'] = de['TPoints'] / de['TTOI']
de['TOIGoalsF'] = de['TGoalsF'] / de['TTOI']
de['TOIGoalsA'] = de['TGoalsA'] / de['TTOI']
de['TOIEVGoalsF'] = de['TEVGoalsF'] / de['TTOI']
de['TOIEVGoalsA'] = de['TEVGoalsA'] / de['TTOI']
de['TOIPlus/Minus'] = de['TPlus/Minus'] / de['TTOI']
de['TOIShots'] = de['TShots'] / de['TTOI']
de['TOIShotsF'] = de['TShotsF'] / de['TTOI']
de['TOIShotsA'] = de['TShotsA'] / de['TTOI']
de['TOIDShots'] = de['TDShots'] / de['TTOI']
de['TOIEVShotsF'] = de['TEVShotsF'] / de['TTOI']
de['TOIEVShotsA'] = de['TEVShotsA'] / de['TTOI']
de['TOIEVDShots'] = de['TEVDShots'] / de['TTOI']
de['TOIBlocks'] = de['TBlocks'] / de['TTOI']
de['TOIHits'] = de['THits'] / de['TTOI']
de['TOIPenalties'] = de['TPenalties'] / de['TTOI']
de['TOITakeaways'] = de['TTakeaways'] / de['TTOI']
de['TOIGiveaways'] = de['TGiveaways'] / de['TTOI']
de['TOIMisses'] = de['TMisses'] / de['TTOI']

- keep sesaon total variables and scaled variables

In [149]:
de = de[['Season', 'TeamCode', 'PlayerName', 'Position', 'TGP', 'TGoals', 'TAssists', 'TPoints', 'TGoalsF', 'TGoalsA', 'TEVGoalsF', 'TEVGoalsA', 'TPlus/Minus', 'TShots', 'TShotsF', 'TShotsA', 'TDShots', 'TEVShotsF', 'TEVShotsA', 'TEVDShots', 'TBlocks', 'THits', 'TPenalties', 'TTakeaways', 'TGiveaways', 'TMisses', 'TTOI', 'TOIGoals', 'TOIAssists', 'TOIPoints', 'TOIGoalsF', 'TOIGoalsA', 'TOIEVGoalsF', 'TOIEVGoalsA', 'TOIPlus/Minus', 'TOIShots', 'TOIShotsF', 'TOIShotsA', 'TOIDShots', 'TOIEVShotsF', 'TOIEVShotsA', 'TOIEVDShots', 'TOIBlocks', 'TOIHits', 'TOIPenalties', 'TOITakeaways', 'TOIGiveaways', 'TOIMisses']]
de.head()

,Season,TeamCode,PlayerName,Position,TGP,TGoals,TAssists,TPoints,TGoalsF,TGoalsA,TEVGoalsF,TEVGoalsA,TPlus/Minus,TShots,TShotsF,TShotsA,TDShots,TEVShotsF,TEVShotsA,TEVDShots,TBlocks,THits,TPenalties,TTakeaways,TGiveaways,TMisses,TTOI,TOIGoals,TOIAssists,TOIPoints,TOIGoalsF,TOIGoalsA,TOIEVGoalsF,TOIEVGoalsA,TOIPlus/Minus,TOIShots,TOIShotsF,TOIShotsA,TOIDShots,TOIEVShotsF,TOIEVShotsA,TOIEVDShots,TOIBlocks,TOIHits,TOIPenalties,TOITakeaways,TOIGiveaways,TOIMisses
0,2015,MTL,TOMAS PLEKANEC,C,82,14.0,40.0,54.0,83.0,61.0,58.0,45.0,13.0,175.0,800.0,711.0,89.0,618.0,602.0,16.0,46.0,60.0,36.0,15.0,45.0,61.0,1520.0,0.009211,0.026316,0.035526,0.054605,0.040132,0.038158,0.029605,0.008553,0.115132,0.526316,0.467763,0.058553,0.406579,0.396053,0.010526,0.030263,0.039474,0.023684,0.009868,0.029605,0.040132
1,2015,MTL,BRENDAN GALLAGHER,W,53,19.0,21.0,40.0,65.0,25.0,44.0,25.0,19.0,154.0,555.0,379.0,176.0,420.0,378.0,42.0,24.0,43.0,24.0,30.0,14.0,42.0,879.0,0.021615,0.023891,0.045506,0.073948,0.028441,0.050057,0.028441,0.021615,0.175199,0.631399,0.431172,0.200228,0.477816,0.430034,0.047782,0.027304,0.048919,0.027304,0.034130,0.015927,0.047782
2,2015,MTL,MAX PACIORETTY,W,82,30.0,34.0,64.0,88.0,73.0,61.0,62.0,-1.0,273.0,897.0,691.0,206.0,703.0,611.0,92.0,37.0,100.0,34.0,35.0,33.0,122.0,1519.0,0.019750,0.022383,0.042133,0.057933,0.048058,0.040158,0.040816,-0.000658,0.179724,0.590520,0.454905,0.135616,0.462804,0.402238,0.060566,0.024358,0.065833,0.022383,0.023041,0.021725,0.080316
3,2015,MTL,PK SUBBAN,D,68,6.0,45.0,51.0,107.0,75.0,72.0,60.0,12.0,170.0,1001.0,856.0,145.0,725.0,718.0,7.0,79.0,101.0,75.0,23.0,106.0,72.0,1793.0,0.003346,0.025098,0.028444,0.059677,0.041829,0.040156,0.033463,0.006693,0.094813,0.558282,0.477412,0.080870,0.404350,0.400446,0.003904,0.044060,0.056330,0.041829,0.012828,0.059119,0.040156
4,2015,MTL,ANDREI MARKOV,D,82,5.0,39.0,44.0,98.0,85.0,67.0,64.0,3.0,112.0,992.0,992.0,0.0,754.0,824.0,-70.0,142.0,57.0,38.0,38.0,93.0,58.0,1955.0,0.002558,0.019949,0.022506,0.050128,0.043478,0.034271,0.032737,0.001535,0.057289,0.507417,0.507417,0.000000,0.385678,0.421483,-0.035806,0.072634,0.029156,0.019437,0.019437,0.047570,0.029668


In [150]:
de.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/stats/2015_player_stats.csv', index='False', sep=',')
#de = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/stats/2015_player_stats.csv', index='False', sep=',')